<a href="https://colab.research.google.com/github/sethkipsangmutuba/OOP---Telecommunication-Information-Engineering-Python/blob/main/Note3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Object Similarity, Inheritance, and Polymorphism

#Seth Kipsang

## 1. Concept of Object Similarity
- Structural and behavioral equivalence of objects  
- Role of shared interfaces and behavioral contracts  

## 2. Fundamentals of Inheritance
- Basic inheritance as a mechanism for code reuse  
- Construction of type hierarchies and conceptual generalisation  

## 3. Extension of Built-in Types
- Inheriting from and adapting built-in classes  
- Implications for semantic correctness, robustness, and long-term maintainability  

## 4. Method Overriding and `super` Semantics
- Redefinition of inherited behavior  
- Controlled delegation to parent implementations using `super`  

## 5. Multiple Inheritance
- Motivations for multiple inheritance  
- Expressiveness versus structural and cognitive complexity  

## 6. The Diamond Problem
- Ambiguity in method resolution  
- Linearization strategies and formal method resolution models  

## 7. Heterogeneous Argument Sets in Inheritance Hierarchies
- Method signatures across class hierarchies  
- Substitutability principles and variance constraints  

## 8. Polymorphism
- Dynamic binding and late dispatch  
- Interface-based design and behavioral substitution  

## 9. Abstract Base Classes (ABCs)
- Role of abstraction in enforcing behavioral contracts  
- Separation of specification from implementation  

## 10. Applying Abstract Base Classes
- Using ABCs to structure extensible systems  
- Supporting verification, consistency, and controlled extensibility  

## 11. Designing Abstract Base Classes
- Principles for defining abstract interfaces  
- Preservation of invariants and minimal assumptions  

## 12. Demystifying Object-Oriented Mechanisms
- Runtime method resolution  
- Dispatch tables and object layout considerations  

## 13. Case Study
- Analysis of an object-oriented system  
- Inheritance and polymorphism trade-offs in real-world design  

## 14. Exercises
- Research-oriented problems  
- Focus on design correctness, abstraction limits, and extensibility  

## 15. Summary
- Key takeaways on inheritance, polymorphism, and abstraction  
- Conceptual and practical implications for object-oriented systems  


## When Objects Are Alike

In object-oriented systems, code duplication is a structural and maintenance risk, particularly in large-scale telecommunication software. Object similarity is addressed through inheritance, which formalizes *is-a* relationships by abstracting shared behavior into superclasses and delegating specialization to subclasses. Inheritance is treated as a behavioral and type-theoretic contract, not merely a reuse mechanism. This lecture examines basic inheritance, inheritance from built-in types, multiple inheritance, and polymorphism with duck typing, emphasizing correctness, substitutability, and scalability in distributed, protocol-driven systems.


#A1) Basic Inheritance

## 1. Basic Inheritance in Python

Every Python class implicitly inherits from the built-in object class. This allows Python to treat all objects uniformly. Explicit inheritance looks like this:


In [265]:
# Explicitly inheriting from object (technically optional in Python 3+)
class MySubClass(object):
    pass


**Superclass / Parent class:** The class being inherited from (`object` in this case).

**Subclass / Derived class:** The class that inherits (`MySubClass`).

**Key concept:** Subclasses can extend or override the behavior of the parent class.

## 2. Using Class Variables

Class variables are shared across all instances of the class. For example, tracking all contacts:


In [266]:
class Contact:
    # Class variable shared by all instances
    all_contacts = []

    def __init__(self, name, email):
        self.name = name      # Instance variable
        self.email = email    # Instance variable
        Contact.all_contacts.append(self)  # Add instance to shared list

# Testing Contact class
c1 = Contact("Alice", "alice@example.com")
c2 = Contact("Bob", "bob@example.com")

print("All contacts:", Contact.all_contacts)
# Output: All contacts: [<Contact object>, <Contact object>]


All contacts: [<__main__.Contact object at 0x78299f929ee0>, <__main__.Contact object at 0x78299f9dd040>]


**Important Note:**

- Accessing `self.all_contacts` can create an instance variable if assigned.
- Always use `ClassName.variable` to access the shared class variable when modifying it.

## 3. Extending Functionality with Inheritance

Instead of adding specialized behavior to `Contact` (which might break abstraction), we create a subclass for `Supplier`:


In [267]:
class Supplier(Contact):
    def order(self, order):
        """Send an order to the supplier"""
        print(f"If this were a real system we would send '{order}' order to '{self.name}'")


- `Supplier` inherits everything from `Contact` (`name`, `email`, and registration in `all_contacts`).
- `Supplier` adds new functionality (`order`) that only applies to suppliers.

## 4. Using the Classes


In [268]:
# Creating a normal contact
c = Contact("Some Body", "somebody@example.net")

# Creating a supplier
s = Supplier("Sup Plier", "supplier@example.net")

# Accessing instance variables
print(c.name, c.email)  # Some Body somebody@example.net
print(s.name, s.email)  # Sup Plier supplier@example.net

# Class variable shows all contacts, including suppliers
print(c.all_contacts)
# [<Contact object>, <Supplier object>]

# Method demonstration
try:
    c.order("I need pliers")
except AttributeError as e:
    print("Error:", e)
# Error: 'Contact' object has no attribute 'order'

s.order("I need pliers")
# Output: If this were a real system we would send 'I need pliers' order to 'Sup Plier'


Some Body somebody@example.net
Sup Plier supplier@example.net
[<__main__.Contact object at 0x78299f929ee0>, <__main__.Contact object at 0x78299f9dd040>, <__main__.Contact object at 0x78299efe4740>, <__main__.Supplier object at 0x78299e3907d0>]
Error: 'Contact' object has no attribute 'order'
If this were a real system we would send 'I need pliers' order to 'Sup Plier'


## Key Takeaways forUnderstanding

- **Minimal syntax for inheritance:** Python handles object inheritance automatically, but explicit syntax clarifies intent.  
- **Class variables vs. instance variables:** Shared resources vs. per-object data.  
- **Safe extension via subclasses:** Add functionality without modifying base class behavior.  
- **Encapsulation of behavior:** Only the subclass gets methods specific to its role (`Supplier.order`).  
- **Real-world analogy:** `Contact` represents general people; `Supplier` represents specialized actors in a network/system.


#A2) Extending Built-in

## 1. Extending Built-in Types in Python

Python allows us to subclass built-in classes like `list`, `dict`, and `str` to add custom behavior. This is particularly useful when we want to attach domain-specific methods to existing data structures.

### Example 1: Extending `list` for Contact Management

We want to add a search-by-name method to a list of contacts. Instead of placing search logic in the `Contact` class, we extend `list`:


In [269]:
class ContactList(list):
    def search(self, name):
        """Return all contacts whose names contain the search value."""
        matching_contacts = [contact for contact in self if name in contact.name]
        return matching_contacts

class Contact:
    # all_contacts is now a ContactList instead of a plain list
    all_contacts = ContactList()

    def __init__(self, name, email):
        self.name = name
        self.email = email
        # Append the contact to the shared ContactList
        Contact.all_contacts.append(self)

# Example usage
c1 = Contact("John A", "johna@example.net")
c2 = Contact("John B", "johnb@example.net")
c3 = Contact("Jenna C", "jennac@example.net")

# Search contacts by name
results = Contact.all_contacts.search('John')
print([c.name for c in results])
# Output: ['John A', 'John B']


['John A', 'John B']


## Key Insights

- We added domain-specific behavior to a standard `list` without modifying Python itself.  
- `ContactList` inherits all behaviors of `list` (`append`, `remove`, iteration, slicing) and adds a custom search method.  
- This demonstrates inheritance as a mechanism to specialize built-in types.

### Example 2: Extending `dict` for Custom Behavior

Suppose we want a dictionary that can easily return the longest key:


In [270]:
class LongNameDict(dict):
    def longest_key(self):
        """Return the key with the maximum length."""
        longest = None
        for key in self:
            if not longest or len(key) > len(longest):
                longest = key
        return longest

# Example usage
longkeys = LongNameDict()
longkeys['hello'] = 1
longkeys['longest yet'] = 5
longkeys['hello2'] = 'world'

print(longkeys.longest_key())
# Output: 'longest yet'


longest yet


## Key Insights

- `LongNameDict` inherits all standard `dict` behaviors: key lookup, assignment, iteration, and deletion.  
- The `longest_key` method adds domain-specific functionality, tailored to the problem domain.  

## 3. Why This Matters in Practice

- **Encapsulation of domain logic:** Methods like `search` and `longest_key` live on the object that logically owns the data structure, keeping classes modular.  
- **Reuse and maintainability:** Any new list of contacts or dict of key-value pairs automatically gets the custom methods without rewriting logic.  
- **Alignment with OOP principles:**  
  - **Inheritance:** reuse of existing behaviors  
  - **Abstraction:** only expose methods meaningful to the domain (`search`, `longest_key`)  
  - **Polymorphism:** extended lists and dicts still behave like standard Python lists and dicts in generic contexts  

## 4. Advanced Extension Ideas

- Extend `set` to provide union/intersection methods tailored to telecom networks (e.g., overlapping devices).  
- Extend `str` for custom parsing of protocol messages.  
- Extend numeric types (`int`, `float`) for domain-specific calculations (e.g., converting signal strength to dBm).  

This approach sets the stage for a more complex telecom example, where we could have:  

- `DeviceList(ContactList)` → manages IoT or network devices with search, filter, and routing functions.  
- `NetworkMetricsDict(LongNameDict)` → stores performance metrics and allows querying max latency, highest throughput, etc.


#A3) Overriding and super

## 1. Method Overriding and `super()` in Python

When you want to change the behavior of a superclass method in a subclass, you override it by defining a method with the same name. For example, adding a phone number to contacts:


In [271]:
class Contact:
    all_contacts = []

    def __init__(self, name, email):
        self.name = name
        self.email = email
        # Add this contact to the shared list
        Contact.all_contacts.append(self)

# Subclass with new attribute: phone
class Friend(Contact):
    def __init__(self, name, email, phone):
        # Call the original __init__ of Contact to avoid duplicating code
        super().__init__(name, email)
        self.phone = phone


## 2. Why Use `super()`?

Without `super()`, we would have to duplicate initialization code from the parent class:


In [272]:
class Friend(Contact):
    def __init__(self, name, email, phone):
        self.name = name      # duplicate
        self.email = email    # duplicate
        self.phone = phone
        # forgot to add to Contact.all_contacts → bug


**Problems with duplicating code:**

- Maintenance nightmare—any change in `Contact.__init__` requires updating all subclasses.  
- Risk of errors—like forgetting to append to `all_contacts`.  

Using `super()` ensures the subclass correctly inherits initialization behavior while extending it safely.


## 3. Example Usage


In [273]:
# Create a regular contact
c = Contact("Alice", "alice@example.com")

# Create a friend with phone number
f = Friend("Bob", "bob@example.com", "+254712345678")

# Verify attributes
print(c.name, c.email)
# Output: Alice alice@example.com

print(f.name, f.email, f.phone)
# Output: Bob bob@example.com +254712345678

# Check that all_contacts includes both
print([person.name for person in Contact.all_contacts])
# Output: ['Alice', 'Bob']


Alice alice@example.com
Bob bob@example.com +254712345678
['Alice']


## 4. Advanced Notes on `super()`

- `super()` can be called anywhere in a method, not just first.  
- It works for all methods, not just `__init__`. For example, you can override a `send_message()` method and still call the parent version.  
- Especially important in multiple inheritance scenarios, where the Method Resolution Order (MRO) determines which parent method `super()` calls.  

## 5. Extending Behavior Safely

We can also override other methods while still using `super()`:


In [274]:
class Friend(Contact):
    def __init__(self, name, email, phone):
        # Validate email before calling parent
        if "@" not in email:
            raise ValueError("Invalid email address")
        super().__init__(name, email)
        self.phone = phone

    def display_info(self):
        # Override display, but reuse parent's formatting if needed
        base_info = f"{self.name} <{self.email}>"
        return f"{base_info}, Phone: {self.phone}"

# Testing
f = Friend("Charlie", "charlie@example.com", "+254700123456")
print(f.display_info())
# Output: Charlie <charlie@example.com>, Phone: +254700123456


Charlie <charlie@example.com>, Phone: +254700123456


## Key Points

- Validation or pre-processing can happen before calling `super()`.  
- Additional behavior can happen after calling `super()`.  
- Cleanly extends parent behavior without duplication.  

 **Summary**

- Overriding allows a subclass to modify the behavior of any method.  
- `super()` ensures the parent class’s method runs, avoiding code duplication.  
- Works for `__init__` and any other method.  
- Critical for maintaining the integrity of inherited properties and class variables.  
- Essential for complex hierarchies and multiple inheritance scenarios.


##A4) Multiple inheritance

## 1. Multiple Inheritance Using Mixins

Multiple inheritance allows a subclass to inherit from more than one parent class. The safest and most common use is **mixins**: classes designed to provide extra functionality but not meant to exist on their own.

### Example: `MailSender` Mixin


In [275]:
# Mixin to provide email functionality
class MailSender:
    def send_mail(self, message):
        """Send an email to the object's email address"""
        print(f"Sending mail to {self.email}: {message}")


- `MailSender` is not a standalone class; it adds behavior to another class.  
- Mixins avoid duplicating code across multiple classes that need the same functionality.  

## 2. Combining `Contact` with `MailSender`


In [276]:
class Contact:
    all_contacts = []

    def __init__(self, name, email):
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)

# New class inherits from both Contact and MailSender
class EmailableContact(Contact, MailSender):
    pass

# Usage
e = EmailableContact("John Smith", "jsmith@example.net")
print([c.name for c in Contact.all_contacts])  # ['John Smith']

# Test mixin
e.send_mail("Hello, this is a test email")
# Output: Sending mail to jsmith@example.net: Hello, this is a test email


['John Smith']
Sending mail to jsmith@example.net: Hello, this is a test email


## Key Insight

- `EmailableContact` combines state from `Contact` and behavior from `MailSender`.  
- No duplication of the email-sending method across multiple classes.  

## 3. Adding Another Layer: `AddressHolder`

Suppose we want to store an address for some contacts. We can create a separate class:


In [277]:
class AddressHolder:
    def __init__(self, street, city, state, code):
        self.street = street
        self.city = city
        self.state = state
        self.code = code


- Conceptually, `Friend` “has an” `Address`, not “is an” `Address` → **composition** is preferred.  
- For teaching multiple inheritance, we can create a hybrid `FriendWithAddress` class:


In [278]:
class Friend(Contact, AddressHolder):
    def __init__(self, name, email, phone, street, city, state, code):
        # Initialize Contact part
        super().__init__(name, email)
        self.phone = phone
        # Initialize AddressHolder part manually
        AddressHolder.__init__(self, street, city, state, code)


- Notice the manual call to `AddressHolder.__init__` because `super()` only calls the next class in the Method Resolution Order (MRO).  
- This is one reason multiple inheritance can get messy—you need to carefully control initialization.  

## 4. Example Usage


In [279]:
f = Friend(
    "Alice Friend",
    "alice@example.net",
    "+254700123456",
    "123 Main St",
    "Nairobi",
    "Nairobi County",
    "00100"
)

print(f.name, f.email, f.phone)
# Output: Alice Friend alice@example.net +254700123456

print(f.street, f.city, f.state, f.code)
# Output: 123 Main St Nairobi Nairobi County 00100

# Friend is also added to all_contacts
print([c.name for c in Contact.all_contacts])
# Output: ['John Smith', 'Alice Friend']


Alice Friend alice@example.net +254700123456
123 Main St Nairobi Nairobi County 00100
['John Smith', 'Alice Friend']


## 5. Trade-offs & Best Practices

- **Mixins are safe:** Only add behavior, not state. Examples: `MailSender`, `LoggerMixin`.  
- **Avoid multiple inheritance for complex state hierarchies:**  
  - Initialization becomes tricky (`super()` works only in linear MRO order).  
  - Harder to maintain and debug in large systems.  
- **Composition often preferred:**  
  - `Friend` could have an `Address` instance instead of inheriting `AddressHolder`.  
  - Reusable across many unrelated classes (`Companies`, `Buildings`, etc.).  
- **When multiple inheritance is justified:**  
  - Clear, non-conflicting mixins with behavior only.  
  - Known MRO and controlled `super()` usage.  

**Summary**

- Multiple inheritance allows a class to combine functionality from multiple parents.  
- Mixins are a clean, pedagogical way to learn multiple inheritance safely.  
- Composition is often a better choice for “has-a” relationships.  
- Proper initialization requires care; `super()` works for linear chains, but direct parent calls may be needed for additional parents.


#B1) The diamond problem

## 1. Diamond Problem Overview

- **Scenario:** A class inherits from two subclasses that share a common base class.  
- **Problem:** Direct calls to the base class can execute it multiple times.  
- **Solution:** Use `super()` to respect Python’s Method Resolution Order (MRO), ensuring each method is called once in the correct order.  

## 2. Naive Approach: Direct Parent Calls


In [280]:
class BaseClass:
    num_base_calls = 0

    def call_me(self):
        print("Calling method on Base Class")
        self.num_base_calls += 1


class LeftSubclass(BaseClass):
    num_left_calls = 0

    def call_me(self):
        BaseClass.call_me(self)  # direct call
        print("Calling method on Left Subclass")
        self.num_left_calls += 1


class RightSubclass(BaseClass):
    num_right_calls = 0

    def call_me(self):
        BaseClass.call_me(self)  # direct call
        print("Calling method on Right Subclass")
        self.num_right_calls += 1


class Subclass(LeftSubclass, RightSubclass):
    num_sub_calls = 0

    def call_me(self):
        LeftSubclass.call_me(self)
        RightSubclass.call_me(self)
        print("Calling method on Subclass")
        self.num_sub_calls += 1


Test Execution:

In [281]:
s = Subclass()
s.call_me()
print(s.num_sub_calls, s.num_left_calls, s.num_right_calls, s.num_base_calls)


Calling method on Base Class
Calling method on Left Subclass
Calling method on Base Class
Calling method on Right Subclass
Calling method on Subclass
1 1 1 2


- **Problem:** `BaseClass.call_me` is executed twice, which can cause serious bugs if the method performs real work (database writes, network operations, financial transactions).  

## 3. Correct Approach: Using `super()`

`super()` resolves the next method in the MRO (not just the parent class). This ensures each method is called once, in the correct order.


In [282]:
class BaseClass:
    num_base_calls = 0

    def call_me(self):
        print("Calling method on Base Class")
        self.num_base_calls += 1


class LeftSubclass(BaseClass):
    num_left_calls = 0

    def call_me(self):
        super().call_me()  # next in MRO
        print("Calling method on Left Subclass")
        self.num_left_calls += 1


class RightSubclass(BaseClass):
    num_right_calls = 0

    def call_me(self):
        super().call_me()  # next in MRO
        print("Calling method on Right Subclass")
        self.num_right_calls += 1


class Subclass(LeftSubclass, RightSubclass):
    num_sub_calls = 0

    def call_me(self):
        super().call_me()  # automatically respects MRO
        print("Calling method on Subclass")
        self.num_sub_calls += 1


Test Execution:

In [283]:
s = Subclass()
s.call_me()
print(s.num_sub_calls, s.num_left_calls, s.num_right_calls, s.num_base_calls)


Calling method on Base Class
Calling method on Right Subclass
Calling method on Left Subclass
Calling method on Subclass
1 1 1 1


**Explanation:**

- `Subclass.call_me()` calls `super()`, which resolves to `LeftSubclass.call_me()` (first in MRO).  
- `LeftSubclass.call_me()` calls `super()`, which resolves to `RightSubclass.call_me()`.  
- `RightSubclass.call_me()` calls `super()`, which resolves to `BaseClass.call_me()`.  
- Each method is executed exactly once in the order defined by the MRO.  

## 4. Visualizing the Diamond


In [284]:
diamond_diagram = """
        BaseClass
        /      \\
 LeftSubclass  RightSubclass
        \\      /
         Subclass
"""

print(diamond_diagram)



        BaseClass
        /      \
 LeftSubclass  RightSubclass
        \      /
         Subclass



The MRO for Subclass is:

In [285]:
Subclass.mro()
# Output: [Subclass, LeftSubclass, RightSubclass, BaseClass, object]


[__main__.Subclass,
 __main__.LeftSubclass,
 __main__.RightSubclass,
 __main__.BaseClass,
 object]

- `super()` traverses the MRO linearly, avoiding multiple calls to the same method.  

## 5. Key Takeaways

- Direct parent calls in multiple inheritance are dangerous → can lead to duplicated work.  
- `super()` is designed for cooperative multiple inheritance → always follows the MRO.  
- MRO ensures base methods are called only once even in diamond-shaped hierarchies.  
- Python’s `super()` works not only for `__init__`, but for any method, making it essential in complex hierarchies.  

**Best practice:**

- Use `super()` in all methods in multiple inheritance scenarios.  
- Ensure all classes in the hierarchy cooperate by calling `super()` instead of direct parent calls.


#B2) Different sets of arguments

## 1. Strategy for Multiple Inheritance with Different `__init__` Arguments

- Each parent class’s `__init__`:  
  - Provides defaults for its parameters.  
  - Accepts `**kwargs` to capture arguments not used by itself.  
  - Calls `super().__init__(**kwargs)` to pass remaining arguments upward.  

- The subclass:  
  - Accepts its own parameters explicitly.  
  - Accepts `**kwargs` to forward unknown parameters.  
  - Calls `super().__init__(**kwargs)` so that all parents get a chance to initialize.  
  - Optional: Explicitly pass some parameters upward using `super().__init__(param=value, **kwargs)` if another parent needs it.  

## 2. Example: Friend with Contact + AddressHolder


In [286]:
# --- Base class to safely handle kwargs for object ---
class Base:
    def __init__(self, **kwargs):
        # absorb leftover kwargs so object.__init__() never receives arguments
        super().__init__()

# --- Contact class ---
class Contact(Base):
    all_contacts = []

    def __init__(self, name="", email="", **kwargs):
        super().__init__(**kwargs)  # allow next class in MRO to initialize
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)

# --- AddressHolder class ---
class AddressHolder(Base):
    def __init__(self, street="", city="", state="", code="", **kwargs):
        super().__init__(**kwargs)
        self.street = street
        self.city = city
        self.state = state
        self.code = code

# --- Friend class ---
class Friend(Contact, AddressHolder):
    def __init__(self, phone="", **kwargs):
        # Correct kwargs handling: pass phone upward if needed
        kwargs.update({'phone': phone})
        super().__init__(**kwargs)
        self.phone = phone


3. Using the Friend Class

In [287]:
f = Friend(
    name="Alice Friend",
    email="alice@example.com",
    phone="+254700123456",
    street="123 Main St",
    city="Nairobi",
    state="Nairobi County",
    code="00100"
)

# Check all attributes
print(f.name, f.email, f.phone)
print(f.street, f.city, f.state, f.code)

# Check all_contacts list
print([c.name for c in Contact.all_contacts])


Alice Friend alice@example.com +254700123456
123 Main St Nairobi Nairobi County 00100
['Alice Friend']


## 4. Why This Works

- Each `__init__` method only processes the arguments it cares about.  
- Any unknown arguments are passed up the hierarchy automatically via `**kwargs`.  
- The Method Resolution Order (MRO) ensures that each parent class’s `__init__` is called exactly once.  
- No parent class is called twice (avoids diamond problem issues).  

## 5. Tips for Managing `**kwargs`

- Always plan parent constructors with `**kwargs` if you expect multiple inheritance.  
- Document each class: specify which keyword arguments are expected.  
- Use `kwargs.update()` if you need to include subclass-specific parameters before forwarding:


In [288]:
class Friend(Contact, AddressHolder):
    def __init__(self, phone="", **kwargs):  # <-- kwargs declared here
        kwargs.update({'phone': phone})      # <-- now this works
        super().__init__(**kwargs)           # <-- safe call
        self.phone = phone


- Avoid unnecessary multiple inheritance: **Composition** is often simpler and more maintainable.  

 **Summary:**

- Handling different sets of arguments in Python multiple inheritance requires consistent use of `**kwargs`.  
- All parent classes should cooperate by forwarding unknown arguments.  
- Diamond-shaped hierarchies work smoothly if `super()` is used in all classes.  
- Documentation is crucial—otherwise your future self will spend hours figuring out what to pass.


# B3) Polymorphism in Python

## 1. Strategy for Polymorphism

- Polymorphism allows objects of different subclasses (or types) to be used interchangeably when they implement the same interface (methods/attributes).  
- Define a base interface or abstract behavior (optional in Python).  
- Subclasses implement the interface differently.  
- Client code calls methods on the base type without needing to know the concrete subclass.  
- **Duck typing** in Python allows any object that provides the required methods to be used, regardless of inheritance.  

## 2. Example: Audio Files


In [289]:
# --- Base class for audio files ---
class AudioFile:
    def __init__(self, filename):
        if not filename.endswith(self.ext):
            raise Exception("Invalid file format")
        self.filename = filename

# --- MP3 subclass ---
class MP3File(AudioFile):
    ext = "mp3"

    def play(self):
        print(f"playing {self.filename} as mp3")

# --- WAV subclass ---
class WavFile(AudioFile):
    ext = "wav"

    def play(self):
        print(f"playing {self.filename} as wav")

# --- OGG subclass ---
class OggFile(AudioFile):
    ext = "ogg"

    def play(self):
        print(f"playing {self.filename} as ogg")


Polymorphic behavior:

In [290]:
# Initialize files of different types
mp3 = MP3File("song.mp3")
wav = WavFile("tune.wav")
ogg = OggFile("beat.ogg")

# Call play() without caring about the type
for audio in [mp3, wav, ogg]:
    audio.play()


playing song.mp3 as mp3
playing tune.wav as wav
playing beat.ogg as ogg


# B4) Abstract Base Classes (ABCs)

## 1. Strategy for Abstract Base Classes

- ABCs define a **protocol**: a set of methods or properties a class must implement.  
- ABCs are useful when duck typing alone is not enough (e.g., third-party plugins).  
- Any subclass of an ABC must implement all abstract methods and properties to be instantiable.  
- Python provides the `abc` module for defining and using ABCs.  

## 2. Using Existing ABCs (Example: `Container`)


In [291]:
from collections.abc import Container

class OddContainer:
    def __contains__(self, x):
        if not isinstance(x, int) or x % 2 == 0:
            return False
        return True

# Create instance
odd_container = OddContainer()

# Check ABC compliance
print(isinstance(odd_container, Container))   # True
print(issubclass(OddContainer, Container))    # True

# Use 'in' keyword
print(1 in odd_container)  # True
print(2 in odd_container)  # False
print(3 in odd_container)  # True
print("a string" in odd_container)  # False


True
True
True
False
True
False


**Explanation:**

- `__contains__` is the abstract method required by `Container`.  
- Any class implementing `__contains__` automatically behaves like a `Container`.  
- `in` keyword works via `__contains__`—demonstrating how ABCs integrate with Python syntax.  

## 3. Creating Your Own ABC


In [292]:
import abc

class MediaLoader(metaclass=abc.ABCMeta):

    @abc.abstractmethod
    def play(self):
        """Play the media file"""
        pass

    @abc.abstractproperty
    def ext(self):
        """File extension property"""
        pass

    @classmethod
    def __subclasshook__(cls, C):
        """Consider a class a subclass if it implements all abstract methods"""
        if cls is MediaLoader:
            attrs = set(dir(C))
            if set(cls.__abstractmethods__) <= attrs:
                return True
        return NotImplemented


**Explanation:**

- `metaclass=abc.ABCMeta` gives the class abstract capabilities.  
- `@abc.abstractmethod` and `@abc.abstractproperty` enforce implementation in subclasses.  
- `__subclasshook__` allows duck-typed classes (that implement all required methods) to be recognized as subclasses, even if they don’t explicitly inherit.  

## 4. Subclass Behavior


In [293]:
# Fails to instantiate because abstract methods are missing
class Wav(MediaLoader):
    pass

# Uncommenting this will raise TypeError
# w = Wav()  # TypeError: Can't instantiate abstract class Wav with abstract methods ext, play

# Proper subclass with all abstract attributes implemented
class Ogg(MediaLoader):
    ext = ".ogg"
    def play(self):
        print("Playing OGG file")

o = Ogg()  # Works fine
o.play()


Playing OGG file


**Explanation:**

- `Wav` fails because it does not implement `play()` and `ext`.  
- `Ogg` succeeds because it satisfies the ABC’s contract.  
- ABCs provide clear documentation and enforcement for expected interfaces.  

## 5. Why ABCs Matter

- Enforces interface compliance in a Pythonic way.  
- Useful for plugin systems or large frameworks.  
- Complements duck typing by documenting expected behavior.  
- Allows “is-a” relationships without losing flexibility.  

## 6. Summary

- ABCs define a set of required methods and properties.  
- Subclasses must implement all abstract members to be instantiated.  
- `__subclasshook__` enables duck typing for classes that follow the interface.  
- ABCs bridge the gap between strict interface enforcement and Python’s dynamic typing.


# B4) Demystifying the Magic

**Note:** This section explains how Python’s `__subclasshook__` works to allow duck typing with abstract base classes (ABCs). You can make a class recognized as a subclass of an ABC without explicitly inheriting from it, as long as it implements all required methods and properties.

## Step 1: Understand `@classmethod` and `__subclasshook__`

**Notes:**

- `@classmethod` marks a method as callable on the class itself, not just instances.  
- `__subclasshook__(cls, C)` is a special method called by Python when `issubclass(C, cls)` is used.  
- It allows Python to check if a class `C` should be considered a subclass of `cls` based on the presence of required methods, not just inheritance.  

## Step 2: Example Abstract Base Class `MediaLoader`


In [294]:
import abc

class MediaLoader(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def play(self):
        """All media loaders must implement play()."""
        pass

    @property
    @abc.abstractmethod
    def ext(self):
        """All media loaders must provide file extension."""
        pass

    @classmethod
    def __subclasshook__(cls, C):
        """
        Determines if a class C is considered a subclass of MediaLoader
        by checking if it provides all required abstract methods.
        """
        if cls is MediaLoader:
            attrs = set(dir(C))  # Get all attributes and methods of C
            if set(cls.__abstractmethods__) <= attrs:
                return True      # C has all required methods/properties
        return NotImplemented    # Use default behavior otherwise


**Notes on the code:**

- `attrs = set(dir(C))` gets all methods and properties in class `C`.  
- `set(cls.__abstractmethods__) <= attrs` checks if all abstract methods (e.g., `play`, `ext`) exist in `C`.  
- Returns `True` if all are present → Python treats `C` as a subclass.  
- Returns `NotImplemented` otherwise → Python uses normal subclass detection.  

## Step 3: Define a Duck-Typed Class `Ogg`


In [295]:
# This class does NOT explicitly inherit from MediaLoader
class Ogg:
    ext = ".ogg"

    def play(self):
        print("This will play an OGG file!")


**Note:** Python will consider this class a subclass of `MediaLoader` because it has the required methods, even without explicit inheritance.  

## Step 4: Test Subclass Recognition


In [296]:
# Check if Ogg is treated as a subclass
print(issubclass(Ogg, MediaLoader))    # True

# Check if an instance is recognized as MediaLoader
print(isinstance(Ogg(), MediaLoader))  # True

# Call the method
ogg_file = Ogg()
ogg_file.play()  # Output: This will play an OGG file!


True
True
This will play an OGG file!


## Step 5: Key Takeaways

- `__subclasshook__` enables duck-typed subclassing.  
- Classes that implement all abstract methods can be recognized as subclasses without explicit inheritance.  
- Useful for plugins or third-party integrations, where you want to enforce an interface but cannot control inheritance.  
- The combination of ABCs + duck typing allows Python to be flexible while still enforcing method contracts.


# B5) Case Study – Automated Grading System

**Goal:**  
Create a class-based system to let course authors write assignments with:  

- Lesson content  
- Custom answer checking  
- Student tracking and grading  

We will use ABCs, duck typing, and composition.

## Step 1: Define a Simple Course Assignment

**Notes:**  
- Course authors provide lesson content and answer checking.  
- No inheritance required yet.


In [297]:
# Simple example class for Intro to Python assignment
class IntroToPython:
    def lesson(self, student):
        return f"""
Hello {student}. Define two variables:
an integer named a with value 1
and a string named b with value 'hello'
"""
    def check(self, code):
        # Naive check: exact match
        return code.strip() == "a = 1\nb = 'hello'"


## Step 2: Define an Abstract Base Class (ABC)

**Notes:**  
- This enforces that any assignment must implement `lesson` and `check`.  
- We also add a `__subclasshook__` for duck typing.


In [298]:
import abc

class Assignment(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def lesson(self, student):
        pass

    @abc.abstractmethod
    def check(self, code):
        pass

    @classmethod
    def __subclasshook__(cls, C):
        """Allow duck typing: any class with the abstract methods is treated as subclass"""
        if cls is Assignment:
            attrs = set(dir(C))
            if set(cls.__abstractmethods__) <= attrs:
                return True
        return NotImplemented


Test if duck typing works:

In [299]:
print(issubclass(IntroToPython, Assignment))  # True


True


Step 3: Another assignment example

Notes: This one explicitly subclasses Assignment. It calculates averages.

In [300]:
class Statistics(Assignment):
    def lesson(self, student):
        return (
            f"Good work so far, {student}. Now calculate the average of the numbers "
            "1, 5, 18, -3 and assign to a variable named 'avg'"
        )

    def check(self, code):
        import statistics
        code = "import statistics\n" + code
        local_vars = {}
        global_vars = {}
        exec(code, global_vars, local_vars)
        return local_vars.get("avg") == statistics.mean([1, 5, 18, -3])


## Step 4: Create a Grader for a Single Student-Assignment

**Notes:**  
- Composition is used here.  
- Tracks attempts and correct submissions.


In [301]:
class AssignmentGrader:
    def __init__(self, student, AssignmentClass):
        self.assignment = AssignmentClass()
        self.assignment.student = student
        self.attempts = 0
        self.correct_attempts = 0

    def check(self, code):
        self.attempts += 1
        result = self.assignment.check(code)
        if result:
            self.correct_attempts += 1
        return result

    def lesson(self):
        return self.assignment.lesson(self.assignment.student)


## Step 5: Grader Manager Class

**Notes:**  
- Manages multiple assignments and multiple students.  
- Registers assignments via UUIDs.


In [302]:
import uuid

class Grader:
    def __init__(self):
        self.student_graders = {}     # Maps student -> AssignmentGrader
        self.assignment_classes = {}  # Maps UUID -> Assignment class

    def register(self, assignment_class):
        """Register a new assignment class"""
        if not issubclass(assignment_class, Assignment):
            raise RuntimeError("Your class does not have the right methods")
        id = uuid.uuid4()
        self.assignment_classes[id] = assignment_class
        return id

    def start_assignment(self, student, id):
        """Start a new assignment for a student"""
        self.student_graders[student] = AssignmentGrader(
            student, self.assignment_classes[id]
        )

    def get_lesson(self, student):
        return self.student_graders[student].lesson()

    def check_assignment(self, student, code):
        return self.student_graders[student].check(code)

    def assignment_summary(self, student):
        grader = self.student_graders[student]
        return f"""
{student}'s attempts at {grader.assignment.__class__.__name__}:
attempts: {grader.attempts}
correct: {grader.correct_attempts}
passed: {grader.correct_attempts > 0}
"""


## Step 6: Test the System

**Notes:**  
- Connect everything – registration, starting assignments, checking answers, and summaries.


In [303]:
# Initialize grader manager
grader = Grader()

# Register assignments
itp_id = grader.register(IntroToPython)
stat_id = grader.register(Statistics)

# Start IntroToPython for Tammy
grader.start_assignment("Tammy", itp_id)
print("Tammy's Lesson:", grader.get_lesson("Tammy"))

# First naive check (incorrect)
print("Tammy's check:", grader.check_assignment("Tammy", "a = 1 ; b = 'hello'"))

# Correct check
print("Tammy's correct check:", grader.check_assignment("Tammy", "a = 1\nb = 'hello'"))

# Summary
print(grader.assignment_summary("Tammy"))

# Start Statistics assignment
grader.start_assignment("Tammy", stat_id)
print("Tammy's Lesson:", grader.get_lesson("Tammy"))
print("Tammy's check:", grader.check_assignment("Tammy", "avg=5.25"))
print("Tammy's correct check:", grader.check_assignment(
    "Tammy", "avg = statistics.mean([1, 5, 18, -3])"
))
print(grader.assignment_summary("Tammy"))


Tammy's Lesson: 
Hello Tammy. Define two variables:
an integer named a with value 1
and a string named b with value 'hello'

Tammy's check: False
Tammy's correct check: True

Tammy's attempts at IntroToPython:
attempts: 2
correct: 1
passed: True

Tammy's Lesson: Good work so far, Tammy. Now calculate the average of the numbers 1, 5, 18, -3 and assign to a variable named 'avg'
Tammy's check: True
Tammy's correct check: True

Tammy's attempts at Statistics:
attempts: 2
correct: 2
passed: True



## Key Points

- Duck typing + ABCs allows assignments to work even without explicit inheritance.  
- Composition (`AssignmentGrader` inside `Grader`) makes the system flexible.  
- UUID registration ensures each assignment is uniquely identified.  
- Each student can attempt assignments multiple times and progress is tracked.  
- This design is safe, modular, and scalable for real-world online grading systems.


# B6) Exercise 1: Inheritance Hierarchy for a Pet Project

**Goal:**  
Identify objects, shared behaviors, and polymorphic differences in a programming project rather than physical objects.

##  Pick a Domain

- Let’s say we want to model a simple zoo simulation.  

**Base class:** `Animal`  

**Derived classes:** `Mammal`, `Bird`, `Reptile`  

**Polymorphic methods:** `speak()` or `move()`  

**Shared properties:** `name`, `age`  

**Different properties:**  
- `wing_span` (for birds)  
- `num_legs` (for most)  
- `scale_type` (for reptiles)  




In [304]:
# ------------------------------
# Base Class: Animal
# ------------------------------
class Animal:
    """Base class for all animals."""
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def speak(self):
        """Default speak method, to be overridden."""
        return f"{self.name} makes a sound."

    def move(self):
        """Default move method, to be overridden."""
        return f"{self.name} moves in some way."

# ------------------------------
# Mixin Class: Swimmer
# ------------------------------
class Swimmer:
    """Mixin class for swimming animals."""
    def swim(self):
        return f"{self.name} is swimming."

# ------------------------------
# Subclass: Mammal
# ------------------------------
class Mammal(Animal):
    def speak(self):
        return f"{self.name} says hello."

    def move(self):
        return f"{self.name} walks or runs."

# ------------------------------
# Subclass: Bird
# ------------------------------
class Bird(Animal):
    def __init__(self, name, age, wing_span=0):
        super().__init__(name, age)
        self.wing_span = wing_span

    def speak(self):
        return f"{self.name} chirps."

    def move(self):
        return f"{self.name} flies with wingspan {self.wing_span} cm."

# ------------------------------
# Subclass: Reptile
# ------------------------------
class Reptile(Animal):
    def __init__(self, name, age, scale_type="scaly"):
        super().__init__(name, age)
        self.scale_type = scale_type

    def speak(self):
        return f"{self.name} hisses."

    def move(self):
        return f"{self.name} slithers on its belly."

# ------------------------------
# Subclass: Duck (Multiple Inheritance)
# ------------------------------
class Duck(Bird, Swimmer):
    def speak(self):
        return f"{self.name} quacks."

    def move(self):
        return f"{self.name} walks, swims, and flies!"

# ------------------------------
# Testing the hierarchy
# ------------------------------
animals = [
    Mammal("Elephant", 10),
    Bird("Parrot", 2, wing_span=25),
    Reptile("Snake", 5, scale_type="smooth"),
    Duck("Donald", 3, wing_span=50)
]

# Loop through animals and show their behavior
for animal in animals:
    print(f"Name: {animal.name}, Age: {animal.age}")
    print("Speak:", animal.speak())
    print("Move:", animal.move())
    # Check if animal can swim
    if isinstance(animal, Swimmer):
        print("Swim:", animal.swim())
    print("-" * 40)


Name: Elephant, Age: 10
Speak: Elephant says hello.
Move: Elephant walks or runs.
----------------------------------------
Name: Parrot, Age: 2
Speak: Parrot chirps.
Move: Parrot flies with wingspan 25 cm.
----------------------------------------
Name: Snake, Age: 5
Speak: Snake hisses.
Move: Snake slithers on its belly.
----------------------------------------
Name: Donald, Age: 3
Speak: Donald quacks.
Move: Donald walks, swims, and flies!
Swim: Donald is swimming.
----------------------------------------


## What This Code Demonstrates

- **Polymorphism:** All animals implement `speak()` and `move()` differently.  
- **Multiple inheritance:** `Duck` inherits from both `Bird` and `Swimmer`.  
- **Mixins:** `Swimmer` adds swimming capability without affecting other animals.  
- **Default arguments:** Avoids `TypeError` for missing parameters.  
- **Safe initialization:** `super().__init__()` ensures proper parent initialization.
